In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from matplotlib.lines import Line2D
from nameparser import HumanName
import gender_guesser.detector as gender
from scipy.stats import chi2_contingency
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import ethnicolr
from ethnicolr import pred_census_ln
import re

In [3]:
# File Paths
absolute_path = "C:\\Users\\tykun\\\OneDrive\\Documents\\SchoolDocs\VSCodeProjects\\connectedData\\board_analysis\\"
altered_dataframes = "altered_dataframes\\"
gpt_dataframes = "gpt_dataframes\\"
graphs = "graphs\\"
scripts =  "scripts\\"
board_dataframes = "board_dataframes\\"
temporary = "temporary_data\\"

altered_dataframe_path = f"{absolute_path}{altered_dataframes}"
gpt_dataframe_path = f"{absolute_path}{gpt_dataframes}" 
graph_path = f"{absolute_path}{graphs}"
script_path = f"{absolute_path}{scripts}"
boards_path = f"{absolute_path}{board_dataframes}"
temporary_data_path = f"{absolute_path}{temporary}"

# Valid Years
years = ["1999", "2000", "2005", "2008", "2009", "2013"]

diversity_statistics_path = f"{altered_dataframe_path}diversity_statistics.csv"

In [4]:
# def clean_name(full_name):
#     """
#     Removes specified substrings and patterns from a full name.

#     Args:
#         full_name (str): The full name to clean.

#     Returns:
#         str: The cleaned full name.
#     """
#     if not isinstance(full_name, str):  # Ensure the input is a string
#         return "Unknown"

#     # Substrings to remove
#     substrings_to_remove = ["Rev.", "SJ", "Sister", "Brother", "Father", "OP", "The Very", "Sr.", "O.P.","Very Rev.", "Br.", "Dr.", "Md.", "S.J.", "Very Rev", "M.D.", "O.P", "S.J", "J.R", "Jr.", "Jr ", "III"]

#     # Remove specified substrings
#     for substring in substrings_to_remove:
#         full_name = full_name.replace(substring, "")

#     # Remove any capital letter followed by a period and a space (e.g., "J. ")
#     full_name = re.sub(r'\b[A-Z]\. ', '', full_name)

#     # Strip extra spaces
#     return full_name.strip()

# def get_last_name(full_name):
#     """
#     Extracts the last name from a full name or returns the full name if extraction fails.

#     Args:
#         full_name (str): The full name of a person.

#     Returns:
#         str: The last name, or the full name if last name extraction fails.
#     """
#     full_name = clean_name(full_name)
#     if not full_name or not full_name.strip():  # Check for None or empty strings
#         return str(full_name)
#     full_name = HumanName(full_name)
#     return str(full_name.last) if full_name.last else str(full_name)

# def get_first_name(full_name):
#     """
#     Extracts the first name from a full name or returns the full name if extraction fails.

#     Args:
#         full_name (str): The full name of a person.

#     Returns:
#         str: The first name, or the full name if first name extraction fails.
#     """
#     full_name = clean_name(full_name)
#     if not full_name or not full_name.strip():  # Check for None or empty strings
#         return str(full_name)
#     full_name = HumanName(full_name)
#     return str(full_name.first) if full_name.first else str(full_name)

# # Classifies ethnicity into 'white', 'poc', or 'unknown'
# def classify_ethnicity(ethnicity):
#     """
#     Classifies ethnicity into 'white', 'poc', or 'unknown'.

#     Args:
#         ethnicity (str): The ethnicity label.

#     Returns:
#         str: The classified ethnicity group.
#     """
#     if ethnicity == 'white':
#         return 'white'
#     elif ethnicity in ['api', 'black', 'hispanic', 'aian']:
#         return 'poc'
#     else:
#         return 'unknown'

# # Classifies gender into 'male', 'female', or 'unknown'
# def classify_gender(gender_str):
#     """
#     Classifies gender into 'male', 'female', or 'unknown'.

#     Args:
#         gender_str (str): The raw gender prediction string.

#     Returns:
#         str: The classified gender group.
#     """
#     if gender_str in ['male', 'mostly_male']:
#         return 'male'
#     elif gender_str in ['female', 'mostly_female']:
#         return 'female'
#     else:
#         return 'unknown'

# # Predicts gender based on first names and adds a 'gender' column to the dataframe
# def pred_gender_fn(df, first_name_col):
#     """
#     Predicts gender based on first names and classifies it.

#     Args:
#         df (pd.DataFrame): The dataframe containing the first name column.
#         first_name_col (str): The column name containing first names.

#     Returns:
#         pd.DataFrame: The dataframe with a new 'gender' column.
#     """
#     d = gender.Detector()
#     df['gender'] = df[first_name_col].apply(lambda x: d.get_gender(x))
#     df['gender'] = df['gender'].apply(classify_gender)
#     return df


# def pred_census_ln_fn(df, last_name_col, year=2000):
#     """
#     Predicts ethnicity based on last names and classifies it.

#     Args:
#         df (pd.DataFrame): The dataframe containing the last name column.
#         last_name_col (str): The column name containing last names.
#         year (int): The year of the Census model to use for predictions.

#     Returns:
#         pd.DataFrame: The dataframe with a new 'ethnicity' column.
#     """
#     # Ensure the last name column contains valid strings
#     df[last_name_col] = df[last_name_col].fillna("Unknown").astype(str)

#     # Sanitize the last_name_col: Remove non-alphanumeric characters and trim whitespace
#     df[last_name_col] = df[last_name_col].apply(lambda x: ''.join(filter(str.isalnum, x)) if isinstance(x, str) else "Unknown")

#     # Use Ethnicolr to predict ethnicity
#     try:
#         df = pred_census_ln(df, last_name_col, year=year)
#     except ValueError as e:
#         print(f"Error in Ethnicolr prediction: {e}")
#         raise

#     # Extract the raw ethnicity prediction column
#     ethnicity_col = f"{last_name_col}_pred_census_{year}"
#     if ethnicity_col not in df.columns:
#         raise ValueError(f"Expected column '{ethnicity_col}' missing after Ethnicolr prediction.")

#     # Create the 'ethnicity' column and classify it
#     df['ethnicity'] = df[ethnicity_col].apply(classify_ethnicity)

#     # Drop the raw prediction column to keep only the classified 'ethnicity'
#     df.drop(columns=[ethnicity_col], inplace=True)

#     return df


# def classify_president_genders(valid_years, altered_dataframe_path):
#     """
#     Classifies gender for each president in the specified years and adds a 'female_president' column.

#     Args:
#         valid_years (list): A list of years to process.
#         altered_dataframe_path (str): The path to the directory containing president CSV files.

#     Returns:
#         dict: A dictionary with 'Institution' as keys and 'female_president' as boolean values.
#     """
#     president_data = {}

#     for year in valid_years:
#         president_df_path = f"{altered_dataframe_path}{year}_presidents.csv"
#         president_df = pd.read_csv(president_df_path)

#         # Extract first names and predict gender
#         president_df['first_name'] = president_df['Name'].apply(get_first_name)
#         president_df['gender'] = president_df['first_name'].apply(lambda x: gender.Detector().get_gender(x))
#         president_df['gender'] = president_df['gender'].apply(classify_gender)

#         # Add 'female_president' column
#         president_df['female_president'] = president_df['gender'].apply(lambda x: True if x == 'female' else False)

#         # Store results in a dictionary by Institution
#         for _, row in president_df.iterrows():
#             president_data[row['Institution']] = row['female_president']

#     return president_data



# def process_board_data():
#     all_years_data = []
#     for year in years:
#         president_df = pd.read_csv(f'{altered_dataframe_path}{year}_presidents.csv')
#         board_df = pd.read_csv(f'{boards_path}{year}_boards.csv')
#         all_years_data.append(board_df)
#         institutions_df = pd.read_csv(f'{gpt_dataframe_path}{year}_gptDataframe.csv')
#         institutions_df = institutions_df[['Institution', 'AffiliationId']].drop_duplicates()

#         #extract gender + ethnicity of the presidents
#         president_df['first_name'] = president_df['Name'].apply(get_first_name)
#         president_df['last_name'] = president_df['Name'].apply(get_last_name)
#         president_df = pred_census_ln(president_df, 'last_name')
#         president_df = pred_gender_fn(president_df, 'first_name')

#         president_df['female_president'] = president_df['gender'].apply(lambda x: True if x == 'female' else False)
#         president_df['poc_president'] = president_df['race'].apply(lambda x: True if x in ['api', 'black', 'hispanic', 'aian'] else False)
#         president_df = president_df.drop_duplicates(subset='Institution').reset_index(drop=True)

#         if president_df['Institution'].is_unique:
#             president_data = president_df.set_index('Institution')[['female_president', 'poc_president', 'AffiliationId']].to_dict('index')
#         else:
#             raise ValueError("Duplicate 'Institution' entries remain after deduplication.")


#         # Extract first and last names for board members
#         board_df['first_name'] = board_df['Name'].apply(get_first_name)
#         board_df['last_name'] = board_df['Name'].apply(get_last_name)



#         #deprecated for now because im not really using it and it wont work
#         # Predict ethnicity using the last name
#         # board_df = pred_census_ln_fn(board_df, 'last_name', 2000)

#         # # Predict gender using the first name
#         board_df = pred_gender_fn(board_df, 'first_name')

#         # # Classify ethnicity and gender
#         # board_df['ethnicity_group'] = board_df['race'].apply(classify_ethnicity)
#         board_df['gender_group'] = board_df['gender']
        

#         # Compute gender counts
#         gender_counts = board_df.groupby(['Institution', 'carnegie_id', 'AffiliationId', 'gender_group']).size().unstack('gender_group', fill_value=0).reset_index()

#         # Merge ethnicity and gender counts
#         counts_df = pd.merge(gender_counts, on=['Institution', 'carnegie_id', 'AffiliationId'], how='outer')

#         # Add a column for total members
#         counts_df['total_members'] = counts_df[['male', 'female', 'unknown']].sum(axis=1)

#         # Add 'Year' column
#         counts_df['Year'] = year

#         counts_df['female_president'] = counts_df['Institution'].map(lambda inst: president_data.get(inst, {}).get('female_president', False))
#         counts_df['AffiliationId'] = counts_df['Institution'].map(lambda inst: president_data.get(inst, {}).get('AffiliationId', ''))

#         cols = ['Year', 'Institution', 'carnegie_id', 'AffiliationId', 'female_president', 'total_members'] + [col for col in counts_df.columns if col not in ['Year', 'Institution', 'carnegie_id', 'AffiliationId', 'female_president', 'total_members']]
#         counts_df = counts_df[cols]
#         # print(board_df.head())
#         all_years_data.append(counts_df)

        
#     all_years_data_df = pd.concat(all_years_data, ignore_index=True)

#     all_years_data_df.fillna(0, inplace=True)

#     # Sort values by 'Institution', 'carnegie_id', 'AffiliationId', and 'Year'
#     all_years_data_df.sort_values(by=['Institution', 'carnegie_id', 'AffiliationId', 'Year'], inplace=True)

#     # Calculate yearly changes for each ethnicity and gender group
#     # ethnicity_columns = ['poc', 'white', 'unknown']  # Adjust based on actual column names
#     gender_columns = ['male', 'female', 'unknown']   # Adjust based on actual column names

#     for col in gender_columns:
#         # Calculate the difference from the previous year for each institution and affiliation
#         all_years_data_df[col + '_change'] = all_years_data_df.groupby(['Institution', 'AffiliationId'])[col].diff()

#     # Sort values for clarity
#     all_years_data_df.sort_values(by=['Year', 'Institution'], inplace=True)
#     return all_years_data_df
        



In [5]:
def clean_name(full_name):
    """
    Removes specified substrings and patterns from a full name.

    Args:
        full_name (str): The full name to clean.

    Returns:
        str: The cleaned full name.
    """
    if not isinstance(full_name, str):  # Ensure the input is a string
        return "Unknown"

    # Substrings to remove
    substrings_to_remove = ["Rev.", "SJ", "Sister", "Brother", "Father", "OP", "The Very", "Sr.", "O.P.","Very Rev.", "Br.", "Dr.", "Md.", "S.J.", "Very Rev", "M.D.", "O.P", "S.J", "J.R", "Jr.", "Jr ", "III"]

    # Remove specified substrings
    for substring in substrings_to_remove:
        full_name = full_name.replace(substring, "")

    # Remove any capital letter followed by a period and a space (e.g., "J. ")
    full_name = re.sub(r'\b[A-Z]\. ', '', full_name)

    # Strip extra spaces
    return full_name.strip()

def get_last_name(full_name):
    """
    Extracts the last name from a full name or returns the full name if extraction fails.

    Args:
        full_name (str): The full name of a person.

    Returns:
        str: The last name, or the full name if last name extraction fails.
    """
    full_name = clean_name(full_name)
    if not full_name or not full_name.strip():  # Check for None or empty strings
        return str(full_name)
    full_name = HumanName(full_name)
    return str(full_name.last) if full_name.last else str(full_name)

def get_first_name(full_name):
    """
    Extracts the first name from a full name or returns the full name if extraction fails.

    Args:
        full_name (str): The full name of a person.

    Returns:
        str: The first name, or the full name if first name extraction fails.
    """
    full_name = clean_name(full_name)
    if not full_name or not full_name.strip():  # Check for None or empty strings
        return str(full_name)
    full_name = HumanName(full_name)
    return str(full_name.first) if full_name.first else str(full_name)

# Classifies ethnicity into 'white', 'poc', or 'unknown'
def classify_ethnicity(ethnicity):
    """
    Classifies ethnicity into 'white', 'poc', or 'unknown'.

    Args:
        ethnicity (str): The ethnicity label.

    Returns:
        str: The classified ethnicity group.
    """
    if ethnicity == 'white':
        return 'white'
    elif ethnicity in ['api', 'black', 'hispanic', 'aian']:
        return 'poc'
    else:
        return 'unknown'

# Classifies gender into 'male', 'female', or 'unknown'
def classify_gender(gender_str):
    """
    Classifies gender into 'male', 'female', or 'unknown'.

    Args:
        gender_str (str): The raw gender prediction string.

    Returns:
        str: The classified gender group.
    """
    if gender_str in ['male', 'mostly_male']:
        return 'male'
    elif gender_str in ['female', 'mostly_female']:
        return 'female'
    else:
        return 'unknown'

# Predicts gender based on first names and adds a 'gender' column to the dataframe
def pred_gender_fn(df, first_name_col):
    """
    Predicts gender based on first names and classifies it.

    Args:
        df (pd.DataFrame): The dataframe containing the first name column.
        first_name_col (str): The column name containing first names.

    Returns:
        pd.DataFrame: The dataframe with a new 'gender' column.
    """
    d = gender.Detector()
    df['gender'] = df[first_name_col].apply(lambda x: d.get_gender(x))
    df['gender'] = df['gender'].apply(classify_gender)
    return df


def pred_census_ln_fn(df, last_name_col, year=2000):
    """
    Predicts ethnicity based on last names and classifies it.

    Args:
        df (pd.DataFrame): The dataframe containing the last name column.
        last_name_col (str): The column name containing last names.
        year (int): The year of the Census model to use for predictions.

    Returns:
        pd.DataFrame: The dataframe with a new 'ethnicity' column.
    """
    # Ensure the last name column contains valid strings
    df[last_name_col] = df[last_name_col].fillna("Unknown").astype(str)

    # Sanitize the last_name_col: Remove non-alphanumeric characters and trim whitespace
    df[last_name_col] = df[last_name_col].apply(lambda x: ''.join(filter(str.isalnum, x)) if isinstance(x, str) else "Unknown")

    # Use Ethnicolr to predict ethnicity
    try:
        df = pred_census_ln(df, last_name_col, year=year)
    except ValueError as e:
        print(f"Error in Ethnicolr prediction: {e}")
        raise

    # Extract the raw ethnicity prediction column
    ethnicity_col = f"{last_name_col}_pred_census_{year}"
    if ethnicity_col not in df.columns:
        raise ValueError(f"Expected column '{ethnicity_col}' missing after Ethnicolr prediction.")

    # Create the 'ethnicity' column and classify it
    df['ethnicity'] = df[ethnicity_col].apply(classify_ethnicity)

    # Drop the raw prediction column to keep only the classified 'ethnicity'
    df.drop(columns=[ethnicity_col], inplace=True)

    return df


def classify_president_genders(valid_years, altered_dataframe_path):
    """
    Classifies gender for each president in the specified years and adds a 'female_president' column.

    Args:
        valid_years (list): A list of years to process.
        altered_dataframe_path (str): The path to the directory containing president CSV files.

    Returns:
        dict: A dictionary with 'Institution' as keys and 'female_president' as boolean values.
    """
    president_data = {}

    for year in valid_years:
        president_df_path = f"{altered_dataframe_path}{year}_presidents.csv"
        president_df = pd.read_csv(president_df_path)

        # Extract first names and predict gender
        president_df['first_name'] = president_df['Name'].apply(get_first_name)
        president_df['gender'] = president_df['first_name'].apply(lambda x: gender.Detector().get_gender(x))
        president_df['gender'] = president_df['gender'].apply(classify_gender)

        # Add 'female_president' column
        president_df['female_president'] = president_df['gender'].apply(lambda x: True if x == 'female' else False)

        # Store results in a dictionary by Institution
        for _, row in president_df.iterrows():
            president_data[row['Institution']] = row['female_president']

    return president_data



def process_board_data():
    """
    Processes board data for each year to compute ONLY gender counts, predicts president gender,
    and adds 'female_president', 'AffiliationId', and 'PrimarySample' columns. Retains original columns
    and adds membership counts, including 'total_members' and yearly changes.

    Returns:
        pd.DataFrame: A DataFrame with original columns, yearly change statistics, and added
                      president/affiliation data (for gender only).
    """
    numbers_list = []

    for year in years:
        # Load the president data for the current year
        president_df_path = f"{altered_dataframe_path}{year}_presidents.csv"
        president_df = pd.read_csv(president_df_path)

        # Extract names & predict gender
        president_df['first_name'] = president_df['Name'].apply(get_first_name)
        president_df['last_name'] = president_df['Name'].apply(get_last_name)
        president_df = pred_gender_fn(president_df, 'first_name')

        # Mark female_president
        president_df['female_president'] = president_df['gender'].apply(lambda x: True if x == 'female' else False)

        # Remove duplicates so each Institution is unique
        president_df = president_df.drop_duplicates(subset='Institution').reset_index(drop=True)

        # Create dictionary mapping institutions -> {female_president, AffiliationId}
        president_data = president_df.set_index('Institution')[['female_president']].to_dict('index')

        # Load the board data for the current year
        board_df_path = f"{boards_path}{year}_boards.csv"
        board_df = pd.read_csv(board_df_path)

        # Extract names & predict gender
        board_df['first_name'] = board_df['Name'].apply(get_first_name)
        board_df['last_name'] = board_df['Name'].apply(get_last_name)
        board_df = pred_gender_fn(board_df, 'first_name')

        # Check for 'PrimarySample' column
        if 'PrimarySample' in board_df.columns:
            primary_sample_map = board_df.drop_duplicates(subset='Institution').set_index('Institution')['PrimarySample']
        else:
            # If 'PrimarySample' doesn't exist, assume False for all
            primary_sample_map = pd.Series(False, index=board_df['Institution'].unique())

        # Group by Institution and gender
        group_cols = ['Institution', 'AffiliationId']
        gender_counts = (
            board_df.groupby(group_cols + ['gender'])
                    .size()
                    .unstack('gender', fill_value=0)
                    .reset_index()
        )

        # Add total_members from male/female/unknown columns
        for col in ['male', 'female', 'unknown']:
            if col not in gender_counts.columns:
                gender_counts[col] = 0
        gender_counts['total_members'] = gender_counts[['male', 'female', 'unknown']].sum(axis=1)

        # Add 'Year' column
        gender_counts['Year'] = year

        # Map female_president and PrimarySample
        gender_counts['female_president'] = gender_counts['Institution'].map(
            lambda inst: president_data.get(inst, {}).get('female_president', False)
        )
        gender_counts['PrimarySample'] = gender_counts['Institution'].map(primary_sample_map).fillna(False)

        # Reorder columns
        main_cols = ['Year', 'Institution', 'AffiliationId', 'female_president', 'PrimarySample', 'total_members']
        other_cols = [c for c in gender_counts.columns if c not in main_cols]
        counts_df = gender_counts[main_cols + other_cols]

        numbers_list.append(counts_df)

    # Concatenate data from all years
    numbers_df = pd.concat(numbers_list, ignore_index=True)
    numbers_df.fillna(0, inplace=True)

    # Sort
    numbers_df.sort_values(by=['Institution', 'AffiliationId', 'Year'], inplace=True)

    # Calculate yearly changes for gender columns
    gender_columns = ['male', 'female', 'unknown']
    for col in gender_columns:
        numbers_df[col + '_change'] = numbers_df.groupby(['Institution', 'AffiliationId'])[col].diff()

    # Final sort
    numbers_df.sort_values(by=['Year', 'Institution'], inplace=True)

    return numbers_df


In [6]:
university_board_statistics_df = process_board_data()
university_board_statistics_df.to_csv(altered_dataframe_path + "university_board_statistics.csv", index = False)

C:\Users\tykun\AppData\Local\Temp\ipykernel_20260\3316104538.py:254: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gender_counts['PrimarySample'] = gender_counts['Institution'].map(primary_sample_map).fillna(False)
C:\Users\tykun\AppData\Local\Temp\ipykernel_20260\3316104538.py:254: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gender_counts['PrimarySample'] = gender_counts['Institution'].map(primary_sample_map).fillna(False)
C:\Users\tykun\AppData\Local\Temp\ipykernel_20260\3316104538.py:254: FutureWarning: Downcasting object dtype arrays on .fillna, .f

In [7]:
def map_affiliation_id(affiliation_df, university_board_statistics_df):
    count_updates = 0  # Counter for the number of updates

    for index, row in affiliation_df.iterrows():
        institution = row['FullName']
        affiliation_id = row['AffiliationId']
        
        # Get the number of matching rows before updating
        matching_rows = university_board_statistics_df[
            (university_board_statistics_df['Institution'] == institution) & 
            (university_board_statistics_df['AffiliationId'].isna())
        ]
        
        # Update AffiliationId if matches are found
        if not matching_rows.empty:
            count_updates += len(matching_rows)
            university_board_statistics_df.loc[
                (university_board_statistics_df['Institution'] == institution) & 
                (university_board_statistics_df['AffiliationId'].isna()), 'AffiliationId'
            ] = affiliation_id

    # Print the total number of updates made
    print(f"Total AffiliationId entries updated: {count_updates}")

    return university_board_statistics_df

In [8]:
# affiliation_df = pd.read_csv(f"{temporary_data_path}affiliation.csv")
# university_board_statistics_df = map_affiliation_id(affiliation_df, university_board_statistics_df)
# university_board_statistics_df.to_csv(altered_dataframe_path + "university_board_statistics.csv", index = False)